In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import sklearn.multiclass as skmul
import sklearn.metrics as skm
import xgboost
!pip install shap
import shap

     |████████████████████████████████| 564 kB 23.0 MB/s 


In [4]:
%matplotlib auto

Using matplotlib backend: agg


In [7]:
# Importation des Données dans le Google Colab
by_colab = True # Vrai si on execute via Google Colab
if (by_colab):
  from google.colab import files
  files.upload()

Saving data.txt to data.txt


# Training & Internal Validation

In [8]:
def read_files(dataset):
    if dataset == 'FvC':
        dataset = '3Class'
    # On demande le nom des fichiers que l'on a importés 
    if (by_colab):
      file_train = input("Nom du fichier de train: ")
      file_test = input("Nom du fichier de test: ")
      train = pd.read_csv(f"{file_train}")
      test = pd.read_csv(f"{file_test}")
    # Comportement de base
    else :
      train = pd.read_csv(f"../data/{dataset}_Train.csv")
      test = pd.read_csv(f"../data/{dataset}_Test.csv")
    return train, test

In [9]:
# Datasets: 3Class, CPosvCNeg, FvOthers, FvC
from os import getcwd,listdir
dataset = 'CPosvCNeg'
train, test = read_files(dataset)
if dataset == 'FvC':
    train = train[train.Class != 2].reset_index(drop=True)
    test = test[test.Class != 2].reset_index(drop=True)

Nom du fichier de traindata.txt
Nom du fichier de testdata.txt


In [10]:
train.head()

,Time and date','ABPmean','ABPsys','ABPdias','C.O.','HR','PAPmean','PAPsys','PAPdias','PULSE','RESP','SpO2','Tblood','Class'
0,'hh:mm:ss dd/mm/yyyy','mmHg','mmHg','mmHg','lpm','bpm','mmHg','mmHg','mmHg','bpm','bpm','%','degC','disease or non'
1,'[15:36:24 02/03/1995]',89.000,114.000,78.000,4.120,80.000,27.000,34.000,22.000,100.000,15.000,100.000,37.100,0
2,'[15:41:25 02/03/1995]',89.000,114.000,78.000,4.120,80.000,26.000,33.000,22.000,100.000,15.000,100.000,37.100,0
3,'[15:46:26 02/03/1995]',89.000,113.000,78.000,4.120,85.000,25.000,32.000,21.000,101.000,15.000,100.000,37.100,0
4,'[15:51:27 02/03/1995]',89.000,113.000,78.000,4.120,80.000,25.000,33.000,22.000,101.000,15.000,100.000,37.100,0


In [ ]:
test.head()

In [ ]:
train.Class.value_counts()

In [ ]:
test.Class.value_counts()

In [ ]:
columns = [
            'sex_F', 'race_White', 'race_AA', 'race_Other',
            'ethnicity_Hispanic_YN', 'Age', 'patient_class',
            'encounter_type', 'reason_for_visit', 'SBP', 'DBP',
            'Temp_C', 'HR', 'RR', 'SPO2', 'BMI', 'BSA', 'Month',
            'Class'
          ]

In [ ]:
train = train[columns]
test = test[columns]

# 3Class

In [ ]:
model = xgboost.XGBClassifier()
model.fit(train.drop('Class', axis=1), train['Class'])

In [ ]:
print(skm.classification_report(test['Class'], model.predict(test.drop('Class', axis=1))))

In [ ]:
skm.confusion_matrix(test['Class'], model.predict(test.drop('Class', axis=1)))

In [ ]:
skm.plot_confusion_matrix(model, test.drop('Class', axis=1), test['Class'], values_format='d')

In [ ]:
clf = skmul.OneVsRestClassifier(xgboost.XGBClassifier())
y_score = clf.fit(train.drop('Class', axis=1), train['Class']).predict_proba(test.drop('Class', axis=1))

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = skm.roc_curve((test['Class'] == i).astype(int), y_score[:, i])
    roc_auc[i] = skm.auc(fpr[i], tpr[i])

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(3):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = %0.3f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC curve')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.legend(loc='best')
plt.show()

In [ ]:
skm.roc_auc_score(test['Class'], model.predict_proba(test.drop('Class', axis=1)), multi_class='ovr', average='macro')

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1))
shap.summary_plot(shap_values, test.drop('Class', axis=1))

# CPosvCNeg

In [ ]:
model = xgboost.XGBClassifier(random_state=1, learning_rate=0.02, max_depth=4, min_child_weight=1, gamma=1,
                              colsample_bytree=1.0, subsample=0.8, n_estimators=600, objective='binary:logistic')
model.fit(train.drop('Class', axis=1), train['Class'],
          sample_weight=sklearn.utils.class_weight.compute_sample_weight("balanced", train['Class']))

In [ ]:
print(skm.classification_report(test['Class'], model.predict(test.drop('Class', axis=1))))

In [ ]:
skm.confusion_matrix(test['Class'], model.predict(test.drop('Class', axis=1)))

In [ ]:
skm.plot_confusion_matrix(model, test.drop('Class', axis=1), test['Class'], values_format='d')

In [ ]:
skm.plot_roc_curve(model, test.drop('Class', axis=1), test['Class'])

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1))
shap.summary_plot(shap_values, test.drop('Class', axis=1))

In [ ]:
shap.initjs()
# Index of sample to explain individual prediction
k = 100
print('Label: ', test.loc[k, 'Class'])
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1).iloc[k, :])
shap.force_plot(explainer.expected_value, shap_values, test.drop('Class', axis=1).iloc[k, :])

# FvOthers

In [ ]:
model = xgboost.XGBClassifier(random_state=1, learning_rate=0.02, max_depth=4, min_child_weight=1, gamma=1,
                              colsample_bytree=1.0, subsample=0.8, n_estimators=600, objective='binary:logistic')
model.fit(train.drop('Class', axis=1), train['Class'],
          sample_weight=sklearn.utils.class_weight.compute_sample_weight("balanced", train['Class']))

In [ ]:
print(skm.classification_report(test['Class'], model.predict(test.drop('Class', axis=1))))

In [ ]:
skm.confusion_matrix(test['Class'], model.predict(test.drop('Class', axis=1)))

In [ ]:
skm.plot_confusion_matrix(model, test.drop('Class', axis=1), test['Class'], values_format='d')

In [ ]:
skm.plot_roc_curve(model, test.drop('Class', axis=1), test['Class'])

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1))
shap.summary_plot(shap_values, test.drop('Class', axis=1))

In [ ]:
shap.initjs()
# Index of sample to explain individual prediction
k = 100
print('Label: ', test.loc[k, 'Class'])
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1).iloc[k, :])
shap.force_plot(explainer.expected_value, shap_values, test.drop('Class', axis=1).iloc[k, :])

# CvF

In [ ]:
model = xgboost.XGBClassifier(random_state=1,learning_rate=0.02, max_depth=4, min_child_weight=1, gamma=1,
                              colsample_bytree=1.0, subsample=0.8, n_estimators=600, objective='binary:logistic')
model.fit(train.drop('Class', axis=1), train['Class'],
          sample_weight=sklearn.utils.class_weight.compute_sample_weight("balanced", train['Class']))

In [ ]:
print(skm.classification_report(test['Class'], model.predict(test.drop('Class', axis=1))))

In [ ]:
skm.confusion_matrix(test['Class'], model.predict(test.drop('Class', axis=1)))

In [ ]:
skm.plot_confusion_matrix(model, test.drop('Class', axis=1), test['Class'], values_format='d')

In [ ]:
skm.plot_roc_curve(model, test.drop('Class', axis=1), test['Class'])

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1))
shap.summary_plot(shap_values, test.drop('Class', axis=1))

In [ ]:
shap.initjs()
# Index of sample to explain individual prediction
k = 100
print('Label: ', test.loc[k, 'Class'])
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop('Class', axis=1).iloc[k, :])
shap.force_plot(explainer.expected_value, shap_values, test.drop('Class', axis=1).iloc[k, :])

# External Validation

In [1]:
dataset = "FvC_Sample"
df = pd.read_csv(f"../data/{dataset}_External_Validation.csv")

NameError: ignored

In [ ]:
df = df[columns]

In [ ]:
df.Class.value_counts()

In [ ]:
skm.classification_report(df['Class'], model.predict(df.drop('Class', axis=1)))

In [ ]:
skm.confusion_matrix(df['Class'], model.predict(df.drop('Class', axis=1)))

In [ ]:
skm.plot_confusion_matrix(model, df.drop('Class', axis=1), df['Class'])